In [ ]:
import csv
from collections import defaultdict
from datetime import datetime

from pyecharts.charts import Bar, Calendar
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from snapshot_selenium import snapshot
from pyecharts.render import make_snapshot

In [ ]:
import trakt.core
from trakt.movies import Movie
from trakt.tv import TVShow

trakt.core.AUTH_METHOD = trakt.core.OAUTH_AUTH
trakt.core.init("mzfr", client_id="", client_secret="", store=True)

In [ ]:
def get_date_informat(date: str, local=False):
    if local:
        date_str = datetime.strptime(date, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")
    else:
        date_str = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d")
    return datetime.strptime(date_str, "%Y-%m-%d")

# Working with Movie Data

In [ ]:
# In this I took out information for all the years.
# If you are doing this then don't run the above cell because that
# was just for a year

detailed_info = {}

with open("export_movies_history.csv", "r") as f:
    movies = csv.DictReader(f)
    for movie in movies:
        date = get_date_informat(movie["watched_at"])
        code = movie["trakt"]

        if code in detailed_info:
            detailed_info[code]['count'] += 1
        else:
            detailed_info[code] = {}
            detailed_info[code]['count'] = 1
            detailed_info[code]['date'] = date
            
            movie_details = Movie(code)
            detailed_info[code]['runtime'] = movie_details.runtime
            detailed_info[code]['genres'] = movie_details.genres

In [ ]:
# From the above cell you can save(json.dump) the data into a new file 
# so that way you can play with the data without having to do a API call via trakt package

import json

with open('detailed_info.json', 'r') as f:
    detailed_info = json.load(f)

In [ ]:
totalTime = 0
for _, v in detailed_info.items():
    yr = get_date_informat(v['date'], local=True)
    if yr.year == 2022:
        totalTime = totalTime + v['runtime']

print(totalTime)

In [ ]:
# Trying to compare genre's watched from multiple years

yearly_genre = {"2022":{}, "2021": {}, "2020": {}, "2019": {}}
for _, movie in detailed_info.items():
    date = get_date_informat(movie['date'], local=True)
    yr = str(date.year)
    if yr not in ['2022', '2021', '2020', '2019']:
        continue
    for g in movie['genres']:
        if g in yearly_genre[yr]:
            yearly_genre[yr][g] += 1
        else:
            yearly_genre[yr][g] = 1


In [ ]:
def multi_year_formatting(yearly_genre, limit):
    """This function is something bogus. Maybe there is an 
    efficient and clear way to do this but basically I am 
    merging multi year genre information and then removing 
    all those genres whose sum is less then 25, this way we 
    will have a clearner multi year graph

    Args:
        yearly_genre (dict): dictionary of format {'2022': {'genre1':1},'2021': {'genre1':3}}
    """
    dd = defaultdict(list)

    for yr, genre in yearly_genre.items():
        for key, value in genre.items():
            dd[key].append(value)
    
    multi_yr_genres = {}

    for k, val in dd.items():
        if sum(val) >= limit:
            multi_yr_genres[k] = val
    
    return multi_yr_genres

In [ ]:
t1 = []
t2 = []
t3 = []
t4 = []
multi_yr_genres = multi_year_formatting(yearly_genre, 25)
for _, val in multi_yr_genres.items():
      t1.append(val[0])
      t2.append(val[1])
      t3.append(val[2])
      t4.append(val[3])


In [ ]:
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
genre = list(multi_yr_genres.keys())
bar.add_xaxis(genre)
bar.add_yaxis("2022", t1)
bar.add_yaxis("2021", t2)
bar.add_yaxis("2020", t3)
# bar.add_yaxis("2019", t4)

# bar.reversal_axis()
bar.set_series_opts(label_opts=opts.LabelOpts(position=None))
bar.set_global_opts(title_opts=opts.TitleOpts(title="Movies"))
# bar.render()
make_snapshot(snapshot, bar.render(), "movies_genre_multi.png")

# TV Shows graphs

In [ ]:
tvShows = {}
dayWiseTV = {}
with open("export_shows_history.csv", "r") as f:
    shows = csv.DictReader(f)
    for show in shows:
        date = get_date_informat(show["watched_at"])
        code = show["trakt"]
        if date.year not in [2022, 2021]:
            continue
        else:
            if code in tvShows:
                tvShows[code]["count"] += 1
            else:
                tvShows[code] = {}
                tvShows[code]["count"] = 1
                tvShows[code]["date"] = date

                show = TVShow(code)
                tvShows[code]["runtime"] = show.runtime
                tvShows[code]["genres"] = show.genres

            if date in dayWiseTV:
                dayWiseTV[date] += 1
            else:
                dayWiseTV[date] = 1

In [ ]:
# Dump into a file just to be safe

import json
with open('tvShows.json', 'w') as f:
    json.dump(tvShows, f, indent=4, sort_keys=True, default=str)

In [ ]:
totalTvTime = 0
tvGenres = {2022:{}, 2021: {}}
for code, info in tvShows.items():
    yr = info['date'].year
    if "runtime" in info and yr == 2022:
        if int(info["runtime"]) > 60:
            totalTvTime += int(info["runtime"])
        else:
            totalTvTime += (int(info['runtime'])*info["count"])
    if "genres" in info:
        for g in info["genres"]:
            if g in tvGenres[yr]:
                tvGenres[yr][g] += 1
            else:
                tvGenres[yr][g] = 1  

In [ ]:
# Chart for single year

bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
genre = list(tvGenres[2022].keys())
vals = list(tvGenres[2022].values())

bar.add_xaxis(genre)
bar.add_yaxis('TV Shows', vals)

bar.reversal_axis()
bar.set_series_opts(label_opts=opts.LabelOpts(position=None))
# bar.set_global_opts(title_opts=opts.TitleOpts(title="TV Show"))
# bar.render()
make_snapshot(snapshot, bar.render(), "shows_genre_2022.png")

In [ ]:
t1 = []
t2 = []
multi_yr_genres = multi_year_formatting(tvGenres, 15)
print(multi_yr_genres)
for _, val in multi_yr_genres.items():
      t1.append(val[0])
      t2.append(val[1])


In [ ]:
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
genre = list(multi_yr_genres.keys())
bar.add_xaxis(genre)

bar.add_yaxis("2022", t1)
bar.add_yaxis("2021", t2)

bar.reversal_axis()
bar.set_series_opts(label_opts=opts.LabelOpts(position=None))
bar.set_global_opts(title_opts=opts.TitleOpts(title="TV Show"))
# bar.render()
make_snapshot(snapshot, bar.render(), "shows_genre_multi.png")

# Calendar Heatmap for Show WatchTime

In [ ]:
calendar_data = []
for k,v in dayWiseTV.items():
    if k.year == 2022:
        calendar_data.append([str(k).split()[0],v])


calendar_data.reverse()
print(calendar_data)

In [ ]:
cal = Calendar()
cal.add("", calendar_data, calendar_opts=opts.CalendarOpts(range_="2022"))
cal.set_global_opts(
        title_opts=opts.TitleOpts(title="TV Show Heatmap", subtitle="Episodes per day"),
        visualmap_opts=opts.VisualMapOpts(
            max_=40,
            min_=0,
            orient="horizontal",
            is_piecewise=True,
            pos_top="230px",
            pos_left="100px",
        ),
    )
# cal.render()

make_snapshot(snapshot, cal.render(), 'shows_heatmap.png')